In [1]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
connection_string

'DRIVER=FreeTDS;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;'

In [5]:
cursor.execute('''INSERT INTO  FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_ANALYTICAL_RULES
select
A.ORGANIZATION_MASTER_ID,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end Risk_score,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG b 
on a.ORGANIZATION_MASTER_ID = b.ORGANIZATION_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''')
conn.commit()

In [6]:

WEIGHTED_AVG = pd.read_sql_query('''
SELECT ORGANIZATION_MASTER_ID,
PRIORITY_NO,
CASE WHEN SUM(THRESHOLD_FLG)!= 0 THEN SUM(SQUARE(THRESHOLD_FLG))/SUM(THRESHOLD_FLG) 
ELSE 0
END AS FINAL
FROM (
SELECT T.*,PRIORITY_NO FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK)
INNER JOIN 
(
SELECT DISTINCT ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC AS THRESHOLD_FLG 
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES
WHERE RISK_SCORE_STATIC IS NOT NULL

UNION 

SELECT DISTINCT
A.ORGANIZATION_MASTER_ID,
A.RULE_NO,
CASE 
	WHEN C.THRESHOLD1_1 IS NULL THEN 0
	WHEN A.RULE_VALUE < C.THRESHOLD1_1 THEN 0 
	WHEN A.RULE_VALUE = C.THRESHOLD1_1 THEN 1
	WHEN A.RULE_VALUE <= C.THRESHOLD1_2 THEN 2
	WHEN A.RULE_VALUE <= C.THRESHOLD1_3 THEN 3
	WHEN A.RULE_VALUE <= C.THRESHOLD1_4 THEN 4
	WHEN A.RULE_VALUE <= C.THRESHOLD1_5 THEN 5
	WHEN A.RULE_VALUE <= C.THRESHOLD1_6 THEN 6
	WHEN A.RULE_VALUE <= C.THRESHOLD1_7 THEN 7
	WHEN A.RULE_VALUE <= C.THRESHOLD1_8 THEN 8
	WHEN A.RULE_VALUE <= C.THRESHOLD1_9 THEN 9
	WHEN A.RULE_VALUE <= C.THRESHOLD1_10 THEN 10
	ELSE 10
	END THRESHOLD_FLG
	FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES A 
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG B ON A.ORGANIZATION_MASTER_ID = B.ORGANIZATION_MASTER_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD C ON C.PEER_GROUP_ID = B.PEER_GROUP_ID AND A.RULE_NO = C.RULE_NO
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES D ON D.RULE_NO = A.RULE_NO

WHERE A.RISK_SCORE_STATIC IS NULL
)T
ON T.RULE_NO = RLR.RULE_NO

)T
GROUP BY ORGANIZATION_MASTER_ID,PRIORITY_NO
''',conn)

In [8]:
weights = pd.read_sql_query('''Select distinct PRIORITY_NO, WEIGHTAGE as WEIGHTS FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK)''',conn)

In [9]:
weights.dropna(inplace = True)

In [10]:
weights

PRIORITY_NO  WEIGHTS
0            1  0.50000
1            2  0.30000
2            3  0.15000
3            4  0.05000

In [11]:
WEIGHTED_AVG = WEIGHTED_AVG.merge(weights,how = 'inner',on = 'PRIORITY_NO')

In [12]:
WEIGHTED_AVG['WEIGHTED_SCORE'] = WEIGHTED_AVG['FINAL']*WEIGHTED_AVG['WEIGHTS']

In [13]:
WEIGHTED_AVG.drop(columns = ['PRIORITY_NO','FINAL'],inplace = True)

In [15]:
FINAL_DF = WEIGHTED_AVG.groupby(['ORGANIZATION_MASTER_ID']).sum()

In [16]:
FINAL_DF.reset_index(inplace = True)

In [17]:
risk_ORGANIZATION = pd.read_sql_query('''Select ORGANIZATION_MASTER_ID,PEER_GROUP_ID from fincore_db.fincore_analytics.risk_ORGANIZATION_summary_stg''',conn)

In [18]:
risk_ORGANIZATION

PERSON_MASTER_ID        PEER_GROUP_ID
0                   1  Government Sector_0
1                   2  Government Sector_0
2                   3  Government Sector_0
3                   4  Government Sector_0
4                   5  Government Sector_0
..                ...                  ...
420               532             Others_0
421               533             Others_0
422               534     Private Sector_0
423               535     Private Sector_0
424               536     OTHERS_CHANGED_9

[425 rows x 2 columns]

In [20]:
auto_high_df = pd.read_sql_query('''SELECT ORGANIZATION_MASTER_ID FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES RTSSR WITH(NOLOCK)
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK) ON RLR.RULE_NO = RTSSR.RULE_NO 
WHERE RISK_TYPE = 'AUTO HIGH RULES '
''',conn)

In [21]:
auto_high_ORGANIZATION = auto_high_df['ORGANIZATION_MASTER_ID'].to_list()

In [22]:
auto_high_ORGANIZATION

[46, 51, 147]

In [23]:
FINAL_DF = FINAL_DF[~FINAL_DF['ORGANIZATION_MASTER_ID'].isin(auto_high_ORGANIZATION)]

In [24]:
FINAL_DF = FINAL_DF.merge(risk_ORGANIZATION,how = 'inner', on = 'ORGANIZATION_MASTER_ID')

In [25]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['WEIGHTED_SCORE']/FINAL_DF['WEIGHTS']

In [26]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['RISK_SCORE'].apply(lambda x: round(x))

PERSON_MASTER_ID  RISK_SCORE        PEER_GROUP_ID
0                   1     6.08155  Government Sector_0
1                   2     5.17500  Government Sector_0
2                   3     5.17500  Government Sector_0
3                   4     5.17500  Government Sector_0
4                   5     5.17500  Government Sector_0
..                ...         ...                  ...
416               532     0.00000             Others_0
417               533     0.00000             Others_0
418               534     7.23846     Private Sector_0
419               535     6.83720     Private Sector_0
420               536     7.84211     OTHERS_CHANGED_9

[421 rows x 3 columns]

In [28]:
count = 1
#UPDATING RISK_ORGANIZATION
for ind,row in tqdm(FINAL_DF.iterrows()):
    if count%10000 == 0:
        print(row['ORGANIZATION_MASTER_ID'],' HERE')
        conn.commit()
    count += 1

    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION (ORGANIZATION_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,CREATE_DATE)
    values ({},'{}',{},{})'''.format(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],'Getdate()'))

conn.commit()


421it [00:00, 832.71it/s]


In [ ]:
cursor.execute('''Update fincore_db.fincore_analytics.risk_ORGANIZATION set risk_score = 1 where risk_score = 0''')
conn.commit()

In [ ]:
conn.close()

In [29]:
%store -r no_of_records

In [30]:
no_of_records

425

In [31]:
%store -r job_id

job_id

637

In [32]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()


In [33]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [34]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'SUCCESS',NO_OF_RECORD = {no_of_records} where  JOB_RUN_ID ={job_id}''')

cursor.commit()
conn.close()